# Deep Learning Regression Model
---

In [1]:
# ignore warnings
import warnings
warnings.filterwarnings("ignore")

# pandas, numpy, matplotlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Keras
from keras.models import Sequential, load_model
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor

# sci-kit learn
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.externals import joblib

## Data Setup

In [2]:
# read in files as dfs
mpls_df = pd.read_csv('resources/mpls_solar_weather.csv')
olg_df = pd.read_csv('resources/olg_solar_weather.csv')

# randomize rows in df
mpls_df = mpls_df.sample(frac=1).reset_index(drop=True)
olg_df = olg_df.sample(frac=1).reset_index(drop=True)

features = [
    'clouds_all', 'temp_f', 'pressure', 'humidity', 'wind_speed', 'wind_deg', 'hour', 'day_of_year',
    'month', 'sin_day', 'cos_day', 'sin_hour', 'cos_hour', 'sin_month', 'cos_month', 'dl_sec'
]

# parameter to predict
target = 'power_delivered'

# get input dimensions
input_dim = len(features)

## Keras Neural Network

In [5]:
def deep_model():
    
    '''Build model with 2 hidden layers'''
    
    model = Sequential()
    model.add(Dense(units=input_dim * 4, activation='relu', input_dim=input_dim))
    model.add(Dense(units=input_dim * 2, activation='relu'))
    model.add(Dense(units=input_dim, activation='relu'))
    model.add(Dense(units=1))  # 1 output
    
    # compile with mean-squared error loss function
    model.compile(loss='mean_squared_error',
                 optimizer='adam',
                 metrics=['mse'])
    return model

In [9]:
def get_features_target(df):
    '''Takes in df and returns features and target dataframes for training and validation.'''
    X = df[features].copy()
    y = df[target].copy()
    
    return X, y

In [10]:
def k_fold_cv(X, y, k):
    '''Performs k-fold cross-validation on data. Outputs array of r2 score results.'''
    
    kfold = KFold(n_splits=k, random_state=42)
    results = cross_val_score(nn_reg, X, y, cv=kfold, scoring='r2', verbose=2, n_jobs=-1)
    
    return results

In [8]:
# parameters for regressor
model = deep_model
epochs = 300
batch_size = 64

# build pipeline using standard scaler
nn_reg = Pipeline(steps=[
    ('standardize', StandardScaler()),
    ('estimator', KerasRegressor(build_fn=model,
                                   epochs=epochs,
                                   batch_size=batch_size))
])

## 10-Fold Cross-Validation

In [14]:
# dict of solar datasets to loop over
dfs_dict = {'mpls': mpls_df,
       'olg': olg_df}

# dict to store results
cv_results = {}

# perform cross validation on datasets
for key in dfs_dict:
    # get features and target
    X, y = get_features_target(dfs_dict[key])
    
    # perform 10-fold cross-validation, store in dict
    cv_results[key] = k_fold_cv(X, y, 10)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed: 10.1min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  1.5min finished


In [16]:
# print results for each solar setup
for key in dfs_dict:
    print(f"Results for {key} solar setup:")
    print(f"  Model: Deep Learning Neural Net (3 hidden layers), Epochs: {epochs}, Batch Size: {batch_size}")
    print()
    print("10-fold cross-validation:\n  r2 score: %.2f (%.2f)" % \
          (cv_results[key].mean(), cv_results[key].std()))
    print("-------------------------------------\n\n")

Results for mpls solar setup:
  Model: Deep Learning Neural Net (3 hidden layers), Epochs: 300, Batch Size: 64

10-fold cross-validation:
  r2 score: 0.89 (0.01)
-------------------------------------


Results for olg solar setup:
  Model: Deep Learning Neural Net (3 hidden layers), Epochs: 300, Batch Size: 64

10-fold cross-validation:
  r2 score: 0.89 (0.02)
-------------------------------------




## Model Training

In [17]:
# mpls data:
X, y = get_features_target(mpls_df)   # get data
nn_reg.fit(X, y)     # train model
nn_reg.named_steps['estimator'].model.save('mpls_nn_reg.h5')  # save model

Epoch 1/300
19859/19859 [==============================] - 1s 29us/step - loss: 1155058.0467 - mse: 1155058.0000
Epoch 2/300
19859/19859 [==============================] - 0s 25us/step - loss: 372584.4342 - mse: 372584.4375
Epoch 3/300
19859/19859 [==============================] - 1s 26us/step - loss: 275528.4380 - mse: 275528.5625
Epoch 4/300
19859/19859 [==============================] - 0s 24us/step - loss: 245916.5429 - mse: 245916.5938
Epoch 5/300
19859/19859 [==============================] - 0s 25us/step - loss: 231607.0554 - mse: 231607.1250
Epoch 6/300
19859/19859 [==============================] - 1s 25us/step - loss: 221790.2727 - mse: 221790.2812
Epoch 7/300
19859/19859 [==============================] - 0s 24us/step - loss: 214875.3663 - mse: 214875.4531
Epoch 8/300
19859/19859 [==============================] - 0s 24us/step - loss: 208783.7858 - mse: 208783.8438
Epoch 9/300
19859/19859 [==============================] - 0s 25us/step - loss: 203976.7969 - mse: 203976.7656

19859/19859 [==============================] - 0s 22us/step - loss: 129778.9554 - mse: 129778.9062
Epoch 147/300
19859/19859 [==============================] - 0s 23us/step - loss: 129784.2783 - mse: 129784.2812
Epoch 148/300
19859/19859 [==============================] - 0s 22us/step - loss: 128949.4296 - mse: 128949.4297
Epoch 149/300
19859/19859 [==============================] - 0s 22us/step - loss: 128522.0954 - mse: 128522.1562
Epoch 150/300
19859/19859 [==============================] - 0s 22us/step - loss: 129417.6501 - mse: 129417.6875
Epoch 151/300
19859/19859 [==============================] - 0s 22us/step - loss: 128595.2589 - mse: 128595.2109
Epoch 152/300
19859/19859 [==============================] - 0s 22us/step - loss: 128324.6120 - mse: 128324.5703
Epoch 153/300
19859/19859 [==============================] - 0s 22us/step - loss: 128522.4147 - mse: 128522.4531
Epoch 154/300
19859/19859 [==============================] - 0s 22us/step - loss: 128077.6193 - mse: 128077.59

19859/19859 [==============================] - 0s 22us/step - loss: 116226.8152 - mse: 116226.7734
Epoch 291/300
19859/19859 [==============================] - 0s 22us/step - loss: 116042.5451 - mse: 116042.5625
Epoch 292/300
19859/19859 [==============================] - 0s 23us/step - loss: 116734.0767 - mse: 116734.0938
Epoch 293/300
19859/19859 [==============================] - 0s 22us/step - loss: 116642.1307 - mse: 116642.1250
Epoch 294/300
19859/19859 [==============================] - 0s 22us/step - loss: 118168.6844 - mse: 118168.6953
Epoch 295/300
19859/19859 [==============================] - 0s 22us/step - loss: 116357.8042 - mse: 116357.8047
Epoch 296/300
19859/19859 [==============================] - 0s 22us/step - loss: 114973.2867 - mse: 114973.2734
Epoch 297/300
19859/19859 [==============================] - 0s 22us/step - loss: 116246.7971 - mse: 116246.8438
Epoch 298/300
19859/19859 [==============================] - 0s 22us/step - loss: 116183.3968 - mse: 116183.42

In [18]:
# olg data:
X, y = get_features_target(olg_df)   # get data
nn_reg.fit(X, y)     # train model
nn_reg.named_steps['estimator'].model.save('olg_nn_reg.h5')  # save model

Epoch 1/300
2582/2582 [==============================] - 0s 69us/step - loss: 1194011087.6158 - mse: 1194011264.0000
Epoch 2/300
2582/2582 [==============================] - 0s 32us/step - loss: 1193705952.3718 - mse: 1193705856.0000
Epoch 3/300
2582/2582 [==============================] - 0s 31us/step - loss: 1191921746.3424 - mse: 1191921920.0000
Epoch 4/300
2582/2582 [==============================] - 0s 31us/step - loss: 1184260421.6019 - mse: 1184260224.0000
Epoch 5/300
2582/2582 [==============================] - 0s 34us/step - loss: 1161666675.5074 - mse: 1161666688.0000
Epoch 6/300
2582/2582 [==============================] - 0s 32us/step - loss: 1111367415.2750 - mse: 1111367552.0000
Epoch 7/300
2582/2582 [==============================] - 0s 34us/step - loss: 1020469758.2153 - mse: 1020469824.0000
Epoch 8/300
2582/2582 [==============================] - 0s 32us/step - loss: 884375283.5074 - mse: 884375360.0000
Epoch 9/300
2582/2582 [==============================] - 0s 33us/s

2582/2582 [==============================] - 0s 27us/step - loss: 84086275.8513 - mse: 84086288.0000
Epoch 143/300
2582/2582 [==============================] - 0s 25us/step - loss: 83909096.7250 - mse: 83909088.0000
Epoch 144/300
2582/2582 [==============================] - 0s 24us/step - loss: 83891261.9799 - mse: 83891272.0000
Epoch 145/300
2582/2582 [==============================] - 0s 27us/step - loss: 83886784.6321 - mse: 83886768.0000
Epoch 146/300
2582/2582 [==============================] - 0s 29us/step - loss: 83805083.3586 - mse: 83805072.0000
Epoch 147/300
2582/2582 [==============================] - 0s 37us/step - loss: 83711366.0852 - mse: 83711368.0000
Epoch 148/300
2582/2582 [==============================] - 0s 53us/step - loss: 83603289.3819 - mse: 83603288.0000
Epoch 149/300
2582/2582 [==============================] - 0s 33us/step - loss: 83822333.2858 - mse: 83822328.0000
Epoch 150/300
2582/2582 [==============================] - 0s 25us/step - loss: 83530830.3393 

2582/2582 [==============================] - 0s 23us/step - loss: 75970559.7041 - mse: 75970552.0000
Epoch 285/300
2582/2582 [==============================] - 0s 24us/step - loss: 76170923.5198 - mse: 76170920.0000
Epoch 286/300
2582/2582 [==============================] - 0s 23us/step - loss: 76043378.4291 - mse: 76043368.0000
Epoch 287/300
2582/2582 [==============================] - 0s 23us/step - loss: 75874661.7475 - mse: 75874656.0000
Epoch 288/300
2582/2582 [==============================] - 0s 23us/step - loss: 76188754.8350 - mse: 76188752.0000
Epoch 289/300
2582/2582 [==============================] - 0s 22us/step - loss: 75698955.7041 - mse: 75698944.0000
Epoch 290/300
2582/2582 [==============================] - 0s 24us/step - loss: 75793847.2068 - mse: 75793848.0000
Epoch 291/300
2582/2582 [==============================] - 0s 22us/step - loss: 75936220.3377 - mse: 75936208.0000
Epoch 292/300
2582/2582 [==============================] - 0s 23us/step - loss: 75782409.1712 

In [19]:
# clear model from pipeline so pipeline can be saved
nn_reg.named_steps['estimator'] = None

# save pipeline
joblib.dump(nn_reg, 'nn_reg_pipeline.pkl')

['nn_reg_pipeline.pkl']